<a href="https://colab.research.google.com/github/YaokunLin/ActuaryProgrammer/blob/main/naive_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
data = {"intents": 
 [ 
      {"tag": "greetings", 
      "patterns": ["hello", "hey", "hi", "good day", "Greetings", "what's up?", "how is it going?"], 
      "responses": ["Hello!", "Hey!", "What can I do for you?"], 
     },
      
      {"tag": "goodbye", 
       "patterns": ["cya", "See you later", "Goodbye", "I am Leaving", "Have a Good day", "bye", "cao", "see ya"], 
       "responses": ["Sad to see you go :(", "Talk to you later", "Goodbye!"], 
      },
       
      {"tag": "age", 
        "patterns": ["how old", "how old is florian", "what is your age", "how old are you", "age?"], 
        "responses": ["My owner Florian is 21 years old!", "21 years!"], 
       },
        
      {"tag": "name", 
         "patterns": ["what is your name", "what should I call you", "whats your name?", "who are you?", "Can you tell me your name?"],
         "responses": ["You can call me Neural!.", "I'm Neural!", "I'm Neural the assistant of Florian!"], 
        }, 
         
      {"tag": "hobby", 
          "patterns": ["what is your hobby", "what do you like to do in your free time?"], 
          "responses": ["I like swimming"]
      }, 
          ]
}

In [ ]:
import pickle 
import numpy as np 
import nltk 
from nltk.stem import WordNetLemmatizer 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Activation, Dropout 
from tensorflow.keras.optimizers import SGD 
import random

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
intents = data
lemmatizer = WordNetLemmatizer()

words = [] 
classes = [] 
documents = [] 
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']: 
  for pattern in intent['patterns']: 
    word_list = nltk.word_tokenize(pattern) 
    words.extend(word_list) 
    documents.append((word_list, intent[ 'tag'])) 
    if intent['tag'] not in classes: 
      classes.append(intent['tag']) 

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters] 
words = sorted(set(words)) 
classes = sorted(set(classes)) 

In [ ]:
pickle.dump(words, open('words.pk1', 'wb')) 
pickle.dump(classes, open('classes.pkl', 'wb')) 

training = [] 
output_empty = [0] * len(classes) 
for document in documents: 
  bag = [] 
  word_patterns = document[0] 
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns] 
  
  for word in words: bag.append(1) if word in word_patterns else bag.append(0) 
  
  output_row = list(output_empty) 
  output_row[classes.index(document[1])] = 1 
  training.append([bag, output_row]) 

In [ ]:
random.shuffle(training) 
training = np.array(training) 


train_x = list(training[:, 0]) 
train_y = list(training[:, 1]) 
model = Sequential() 

model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(64, activation='relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True) 
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy']) 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1) 
model.save('chatbot_model.model', hist) 

print("Done") 


<ipython-input-5-394cee6d3829>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)
/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
6/6 [==============================] - 1s 2ms/step - loss: 1.6242 - accuracy: 0.2593
Epoch 2/200
6/6 [==============================] - 0s 2ms/step - loss: 1.5222 - accuracy: 0.3704
Epoch 3/200
6/6 [==============================] - 0s 2ms/step - loss: 1.5603 - accuracy: 0.1852
Epoch 4/200
6/6 [==============================] - 0s 2ms/step - loss: 1.3834 - accuracy: 0.4444
Epoch 5/200
6/6 [==============================] - 0s 2ms/step - loss: 1.4191 - accuracy: 0.4444
Epoch 6/200
6/6 [==============================] - 0s 2ms/step - loss: 1.4231 - accuracy: 0.4444
Epoch 7/200
6/6 [==============================] - 0s 2ms/step - loss: 1.3264 - accuracy: 0.5185
Epoch 8/200
6/6 [==============================] - 0s 2ms/step - loss: 1.1758 - accuracy: 0.5185
Epoch 9/200
6/6 [==============================] - 0s 2ms/step - loss: 1.0421 - accuracy: 0.6667
Epoch 10/200
6/6 [==============================] - 0s 3ms/step - loss: 1.0318 - accuracy: 0.7778
Epoch 11/200
6/6 [===========

In [ ]:
def clean_up_sentence(sentence): 
  sentence_words = nltk.word_tokenize(sentence) 
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words] 
  return sentence_words 

def bag_of_words(sentence): 
  sentence_words = clean_up_sentence(sentence) 
  bag = [0] * len(words) 
  for w in sentence_words: 
    for i, word in enumerate(words): 
      if word == w: bag[i] = 1 
  return np.array(bag) 

def get_response(intents_list, intents):
   tag = intents_list[0]["intent"] 
   list_of_intents = intents[ 'intents'] 
   
   for i in list_of_intents: 
     if i['tag'] == tag: 
       result = random.choice(i['responses']) 
       break 
   return result

ERROR_THRESHOLD = 0.25
def predict_class(sentence):
   bow = bag_of_words(sentence) 
   res = model.predict(np.array([bow]))[0] 
   results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD] 

   results.sort(key=lambda x: x[1], reverse=True) 
   return_list = [] 
   for r in results: return_list.append({ 'intent': classes[r[0]], 'probability': str(r[1])}) 
   return return_list 



In [ ]:
print ("GO! Bot is runnin !") 
while True: 
  message = input("") 
  ints = predict_class(message) 
  res = get_response(ints, intents) 
  print(res) 

GO! Bot is runnin !
1/1 [==============================] - 0s 71ms/step
What can I do for you?
1/1 [==============================] - 0s 15ms/step
21 years!
